### Imports

In [136]:
from __future__ import division, print_function

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

import matplotlib.pyplot as plt
import time
import os
import cv2
from PIL import Image

plt.ion()   # interactive mode
%matplotlib inline

### Data

In [48]:
TRAIN_DIR = '../data/boneage-training-dataset/'
TRAIN_LABEL = '../data/train.csv'
FILETYPE = '.png'
use_gpu = torch.cuda.is_available()

### Dataset class

In [109]:
class BoneAgeDataset(Dataset):
    """Bone Age dataset."""

    def __init__(self, root_dir, labels, filetype, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.labels = labels
        self.root_dir = root_dir
        self.transform = transform
        self.filetype = filetype

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, str(self.labels.iloc[idx]['id']) + self.filetype)
        img = Image.open(img_name)
#         image = cv2.imread(img_name)[:,:,::-1]
        label = self.labels.iloc[idx]['boneage']

        if self.transform:
            img = self.transform(img)
            
        sample = {'image': img, 'label': label}

        return sample

### Transforms

In [110]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Scale(1024),
        transforms.RandomSizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ]),
    'val': transforms.Compose([
        transforms.Scale(1024),
        transforms.CenterCrop(224),
        transforms.ToTensor()
    ]),
}

### Data loaders

In [111]:
labels = pd.read_csv(TRAIN_LABEL)
bins = xrange(20)
labels['bins'] = np.digitize(labels['boneage'], bins=bins)

In [112]:
skf = StratifiedKFold(n_splits=4)
splits = []
for train, test in skf.split(np.zeros(len(labels)), labels['bins']):
    fold_label = {
        'train': labels.loc[train, ('id', 'boneage')],
        'val': labels.loc[test, ('id', 'boneage')]
    }
    splits.append(fold_label)

In [113]:
fold_label = splits[0]

In [124]:
image_datasets = {x: BoneAgeDataset(TRAIN_DIR, fold_label[x], FILETYPE, data_transforms[x])
                  for x in ['train', 'val']}
dataloders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=16)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

In [131]:
for i_batch, sample in enumerate(dataloders['train']):
    print(i_batch, sample['image'].size())
    break

0 torch.Size([4, 1, 224, 224])


In [149]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(530, 75)
        self.fc2 = nn.Linear(75, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 530)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)
    
model = Net()
if use_gpu:
    model.cuda()

In [150]:
lr = 0.01
momentum = 0.9
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [155]:
def train(epoch, train_loader):
    model.train()
    for batch_idx, sample in enumerate(train_loader):
        data = sample['image']
        target = sample['label']
        if use_gpu:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.l1_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test(test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    for sample in test_loader:
        data = sample['image']
        target = sample['label']
        if use_gpu:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.l1_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [156]:
for epoch in range(1, 10 + 1):
    for phase in ['train', 'val']:
        if phase == 'train':
            train(epoch, dataloders[phase])
        if phase == 'val':
            test(dataloders[phase])

TypeError: CudaAbsCriterion_updateOutput received an invalid combination of arguments - got ([32;1mint[0m, [32;1mtorch.cuda.FloatTensor[0m, [31;1mtorch.cuda.LongTensor[0m, [32;1mtorch.cuda.FloatTensor[0m, [32;1mbool[0m), but expected (int state, torch.cuda.FloatTensor input, torch.cuda.FloatTensor target, torch.cuda.FloatTensor output, bool sizeAverage)